In [1]:
'''
서울시 공원
자치구별 공원면적 단계구분도 
자치구별............그룹
테이블조인.
자치구별 인당 공원면적 단계구분도
================
=주소필드에서 시군구별로 데이터를 그룹핑...피벗이나 그룹바이로 데이터프레임만들고
거기에다 인구정보 두개 테이블 병합하고...작업을 해나가야
============
경기도
시군구별 공원면적 단계구분도
시군구별 
'''

'\n서울시 공원\n자치구별 공원면적 단계구분도 \n자치구별............그룹\n테이블조인.\n자치구별 인당 공원면적 단계구분도\n================\n=주소필드에서 시군구별로 데이터를 그룹핑...피벗이나 그룹바이로 데이터프레임만들고\n거기에다 인구정보 두개 테이블 병합하고...작업을 해나가야\n============\n경기도\n시군구별 공원면적 단계구분도\n시군구별 \n'

In [2]:
import json,folium
import numpy as np
import pandas as pd

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
seoulpark = pd.read_csv('data/서울시 주요 공원현황.csv', encoding='euc-kr')
seoulpark.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   연번            132 non-null    int64  
 1   공원명           132 non-null    object 
 2   공원개요          131 non-null    object 
 3   면적            129 non-null    object 
 4   개원일           122 non-null    object 
 5   주요시설          124 non-null    object 
 6   주요식물          88 non-null     object 
 7   안내도           102 non-null    object 
 8   오시는길          115 non-null    object 
 9   이용시참고사항       120 non-null    object 
 10  이미지           132 non-null    object 
 11  지역            131 non-null    object 
 12  공원주소          132 non-null    object 
 13  관리부서          132 non-null    object 
 14  전화번호          132 non-null    object 
 15  X좌표(GRS80TM)  122 non-null    float64
 16  Y좌표(GRS80TM)  122 non-null    float64
 17  X좌표(WGS84)    131 non-null    float64
 18  Y좌표(WGS84)    131 non-null    

In [5]:
sp = seoulpark[['지역','공원명','면적','공원주소','X좌표(WGS84)','Y좌표(WGS84)']]
sp.columns =['자치구','공원명', '면적', '주소', '경도', '위도']
sp.head()

,자치구,공원명,면적,주소,경도,위도
0,중구,남산도시자연공원,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140
1,강동구,길동생태공원,80683㎡,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394
2,과천시,서울대공원,9132690m²,경기도 과천시 대공원광장로 102,127.019846,37.426449
3,성동구,서울숲,480994㎡,서울특별시 성동구 뚝섬로 273 (성수동1가),127.041798,37.543072
4,마포구,월드컵공원,2284085㎡,서울특별시 마포구 하늘공원로 84(월드컵공원),126.878907,37.571805


In [6]:
# 결측치 제거
sp.dropna(subset=['면적','경도'], inplace=True)
sp.isna().sum() 

자치구    1
공원명    0
면적     0
주소     0
경도     0
위도     0
dtype: int64

In [7]:
# sp.loc[129] 자치구 결측치라서 주소확인하고 노원구 넣어줌
sp['자치구'][129] = '노원구'
sp['자치구'][129]

'노원구'

In [8]:
sp.loc[129]

자치구                노원구
공원명              경춘선숲길
면적             211392㎡
주소     노원구 공릉2동 산 82-2
경도          127.076482
위도           37.627077
Name: 129, dtype: object

In [9]:
# 시작할 때 있는 '총 ' 144m 같은 글에서 '총 ' 지우기
sp.면적 = sp.면적.str.replace('총 ','')

In [10]:
import re 
sp.면적 = sp.면적.apply(lambda x: re.split('[㎡m㎥]', x)[0])
sp.head()

,자치구,공원명,면적,주소,경도,위도
0,중구,남산도시자연공원,2896887,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140
1,강동구,길동생태공원,80683,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394
2,과천시,서울대공원,9132690,경기도 과천시 대공원광장로 102,127.019846,37.426449
3,성동구,서울숲,480994,서울특별시 성동구 뚝섬로 273 (성수동1가),127.041798,37.543072
4,마포구,월드컵공원,2284085,서울특별시 마포구 하늘공원로 84(월드컵공원),126.878907,37.571805


In [11]:
sp.head(10).tail(10)

,자치구,공원명,면적,주소,경도,위도
0,중구,남산도시자연공원,2896887,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140
1,강동구,길동생태공원,80683,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394
2,과천시,서울대공원,9132690,경기도 과천시 대공원광장로 102,127.019846,37.426449
3,성동구,서울숲,480994,서울특별시 성동구 뚝섬로 273 (성수동1가),127.041798,37.543072
4,마포구,월드컵공원,2284085,서울특별시 마포구 하늘공원로 84(월드컵공원),126.878907,37.571805
5,종로구,광화문시민열린마당,8948.1,서울특별시 종로구 세종로 76-2,126.978023,37.575045
6,은평구,진관근린공원(구파발폭포),휴양 및 편익시설 : 평의자 7개 화장실 23.18,서울특별시 은평구 진관동 393,126.919216,37.637024
7,영등포구,중마루근린공원,6456,서울특별시 영등포구 영등포동2가 222,126.911267,37.519832
8,성동구,응봉공원,80309,서울특별시 성동구 금호동1가 산1-72,127.021765,37.557228
9,강서구,매화근린공원,1980.4,서울특별시 강서구 등촌동 630-14,126.861150,37.557583


In [12]:
sp.면적[6]
sp.drop([6], inplace=True)

In [13]:
sp.면적 = sp.면적.astype(float).astype(int)

In [14]:
sp.head()

,자치구,공원명,면적,주소,경도,위도
0,중구,남산도시자연공원,2896887,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140
1,강동구,길동생태공원,80683,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394
2,과천시,서울대공원,9132690,경기도 과천시 대공원광장로 102,127.019846,37.426449
3,성동구,서울숲,480994,서울특별시 성동구 뚝섬로 273 (성수동1가),127.041798,37.543072
4,마포구,월드컵공원,2284085,서울특별시 마포구 하늘공원로 84(월드컵공원),126.878907,37.571805


In [15]:
# 서울 geo_data

In [16]:
filename = 'data/seoul_geo_simple.json'
geo_data = json.load(open(filename, encoding='utf-8'))

In [17]:
geo_data.keys()

dict_keys(['type', 'features'])

In [18]:
geo_data['features'][0]

{'type': 'Feature',
 'id': '강동구',
 'properties': {'code': '11250',
  'name': '강동구',
  'name_eng': 'Gangdong-gu',
  'base_year': '2013'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[127.11519584981606, 37.557533180704915],
    [127.16683184366129, 37.57672487388627],
    [127.18408792330152, 37.55814280369575],
    [127.16530984307447, 37.54221851258693],
    [127.14672806823502, 37.51415680680291],
    [127.12123165719615, 37.52528270089],
    [127.1116764203608, 37.540669955324965],
    [127.11519584981606, 37.557533180704915]]]}}

In [19]:
sp.index

Index([  0,   1,   2,   3,   4,   5,   7,   8,   9,  10,
       ...
       121, 122, 123, 124, 125, 126, 127, 128, 129, 130],
      dtype='int64', length=128)

In [20]:
df = sp.groupby(['자치구'])[['면적']].agg(['sum'])
df.columns = ['면적']

In [21]:
df

,면적
자치구,
강남구,3626398
강동구,1526387
강북구,3714902
강서구,2194152
과천시,9132690
관악구,10431453
광진구,584002
구로구,1617657
금천구,57568


In [22]:
df.index

Index(['강남구', '강동구', '강북구', '강서구', '과천시', '관악구', '광진구', '구로구', '금천구', '노원구',
       '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구',
       '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구'],
      dtype='object', name='자치구')

In [23]:
# df = pd.merge(cctv2, pop3)
# df.set_index('구별', inplace=True)
# df

In [24]:
import numpy as np
def get_text_location(geo_str):
    gu_dict = {}
    for gu in geo_str['features']:
        for coord in gu['geometry']['coordinates']: # 경도,위도 표시
            geo = np.array(coord)
            gu_dict[gu['id']] = [np.mean(geo[:,1]), np.mean(geo[:,0])]  # 경도,위도 표시
    return gu_dict

In [25]:
gu_dict = get_text_location(geo_data) # /seoul_geo_simple
gu_dict

{'강동구': [37.546532751824586, 127.14078218193268],
 '송파구': [37.50620055459428, 127.11197246712628],
 '강남구': [37.49679934694853, 127.0673229896189],
 '서초구': [37.47066332386016, 127.0390912771612],
 '관악구': [37.46393155080159, 126.94996289929526],
 '동작구': [37.495988209152785, 126.9530748356924],
 '영등포구': [37.51883998627326, 126.91028348590825],
 '금천구': [37.46241523438773, 126.9050257773362],
 '구로구': [37.49166908254099, 126.85870943441728],
 '강서구': [37.55703289257873, 126.83102419541086],
 '양천구': [37.529418412364095, 126.85252712500812],
 '마포구': [37.5599165285085, 126.9176866353821],
 '서대문구': [37.57597915089086, 126.94816079153975],
 '은평구': [37.61445865754635, 126.93220977883982],
 '노원구': [37.651705676071316, 127.07821750766884],
 '도봉구': [37.66711228271955, 127.03387331602003],
 '강북구': [37.646476408369075, 127.01356807286824],
 '성북구': [37.61007782092726, 127.02020916322154],
 '중랑구': [37.59847817045754, 127.09036092595524],
 '동대문구': [37.57621738848033, 127.04968775012617],
 '광진구': [37.548640

In [26]:
df.drop(['과천시'],inplace=True)

In [27]:
map = folium.Map([37.55, 126.98], zoom_start=11, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data, # 맵데이터
    data=df.면적, # 단계구분도로 보여줄 데이터
    columns=[df.index, df.면적], # dataframe에서 추출할 컬럼(항목)
    fill_color='YlGnBu',   # Colormap
    key_on='feature.id' # 지도에서 join할 항목
).add_to(map)
for gu_name in df.index:
    folium.map.Marker(
        location=gu_dict[gu_name],
        icon = folium.DivIcon(icon_size=(80,20), icon_anchor=(20,0),
                              html=f'<div style="font-size: 10pt">{gu_name}</div>')
    ).add_to(map)
map
title_html = '<h3 align="center" style="font-size:20px">서울시 자치구별 공원면적</h3>' # 지도 제목달기
map.get_root().html.add_child(folium.Element(title_html))
map

In [28]:
map = folium.Map([37.4, 127.1], zoom_start=9, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=df.면적,      # 단계구분도로 보여줄 데이터
    columns=[df.index, df.면적],    # 데이터프레임에서 추출할 항목
    fill_color='YlOrRd',        # Colormap
    key_on='properties.name'    # 지도에서 조인할 항목
).add_to(map)
map

In [29]:
# 자치구별 인당 공원면적

In [30]:
df

,면적
자치구,
강남구,3626398
강동구,1526387
강북구,3714902
강서구,2194152
관악구,10431453
광진구,584002
구로구,1617657
금천구,57568
노원구,12235699


In [31]:
df.reset_index(inplace=True)
df

,자치구,면적
0,강남구,3626398
1,강동구,1526387
2,강북구,3714902
3,강서구,2194152
4,관악구,10431453
5,광진구,584002
6,구로구,1617657
7,금천구,57568
8,노원구,12235699
9,도봉구,1084218


In [32]:
df.columns =['구별','면적']

In [33]:
df.head()

,구별,면적
0,강남구,3626398
1,강동구,1526387
2,강북구,3714902
3,강서구,2194152
4,관악구,10431453


In [34]:
spop = pd.read_csv('../02.Pandas/data/pop1.csv', encoding='euc-kr')
spop

,구별,인구수,내국인,외국인,고령자,외국인비율,고령자비율
0,종로구,153789,144683,9106,27818,5.92,18.09
1,중구,131787,122499,9288,24392,7.05,18.51
2,용산구,237285,222953,14332,39070,6.04,16.47
3,성동구,292672,285990,6682,46380,2.28,15.85
4,광진구,352627,339996,12631,51723,3.58,14.67
5,동대문구,352006,337400,14606,62211,4.15,17.67
6,중랑구,391885,387350,4535,71682,1.16,18.29
7,성북구,440142,430528,9614,74709,2.18,16.97
8,강북구,302563,299182,3381,64333,1.12,21.26
9,도봉구,319373,317366,2007,64160,0.63,20.09


In [35]:
data1 = pd.merge(df, spop)
data1.set_index('구별', inplace=True)
data1

,면적,인구수,내국인,외국인,고령자,외국인비율,고령자비율
구별,,,,,,,
강남구,3626398,537800,533042,4758,78226,0.88,14.55
강동구,1526387,466472,462664,3808,74070,0.82,15.88
강북구,3714902,302563,299182,3381,64333,1.12,21.26
강서구,2194152,579768,574315,5453,92558,0.94,15.96
관악구,10431453,499449,485699,13750,79871,2.75,15.99
광진구,584002,352627,339996,12631,51723,3.58,14.67
구로구,1617657,421163,396754,24409,72611,5.80,17.24
금천구,57568,244891,230811,14080,41041,5.75,16.76
노원구,12235699,514946,510956,3990,88345,0.77,17.16


In [36]:
data1['인당공원면적'] = (data1.인구수 / data1.면적 *100).round(2)
data1

,면적,인구수,내국인,외국인,고령자,외국인비율,고령자비율,인당공원면적
구별,,,,,,,,
강남구,3626398,537800,533042,4758,78226,0.88,14.55,14.83
강동구,1526387,466472,462664,3808,74070,0.82,15.88,30.56
강북구,3714902,302563,299182,3381,64333,1.12,21.26,8.14
강서구,2194152,579768,574315,5453,92558,0.94,15.96,26.42
관악구,10431453,499449,485699,13750,79871,2.75,15.99,4.79
광진구,584002,352627,339996,12631,51723,3.58,14.67,60.38
구로구,1617657,421163,396754,24409,72611,5.80,17.24,26.04
금천구,57568,244891,230811,14080,41041,5.75,16.76,425.39
노원구,12235699,514946,510956,3990,88345,0.77,17.16,4.21


In [37]:
data1.head()

,면적,인구수,내국인,외국인,고령자,외국인비율,고령자비율,인당공원면적
구별,,,,,,,,
강남구,3626398,537800,533042,4758,78226,0.88,14.55,14.83
강동구,1526387,466472,462664,3808,74070,0.82,15.88,30.56
강북구,3714902,302563,299182,3381,64333,1.12,21.26,8.14
강서구,2194152,579768,574315,5453,92558,0.94,15.96,26.42
관악구,10431453,499449,485699,13750,79871,2.75,15.99,4.79


In [38]:
data2 = data1[['인당공원면적']]


In [39]:
data2.head()

,인당공원면적
구별,
강남구,14.83
강동구,30.56
강북구,8.14
강서구,26.42
관악구,4.79


In [40]:
type(data2.인당공원면적[0])

numpy.float64

In [41]:
map = folium.Map([37.55, 126.98], zoom_start=11, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data, # 맵데이터
    data=data2.인당공원면적, # 단계구분도로 보여줄 데이터
    columns=[data2.index, data2.인당공원면적], # dataframe에서 추출할 컬럼(항목)
    fill_color='YlGnBu',   # Colormap
    key_on='feature.id' # 지도에서 join할 항목
).add_to(map)
for gu_name in data2.index:
    folium.map.Marker(
        location=gu_dict[gu_name],
        icon = folium.DivIcon(icon_size=(80,20), icon_anchor=(20,0),
                              html=f'<div style="font-size: 10pt">{gu_name}</div>')
    ).add_to(map)
map
title_html = '<h3 align="center" style="font-size:20px">서울시 자치구별 인당공원면적</h3>' # 지도 제목달기
map.get_root().html.add_child(folium.Element(title_html))
map